In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
from branca.colormap import linear
from folium.plugins import TimeDynamicGeoJson
import folium
%matplotlib inline

In [ ]:
assert 'naturalearth_lowres' in gpd.datasets.available
datapath = gpd.datasets.get_path('naturalearth_lowres')
gdf = gpd.read_file(datapath)

In [ ]:
gdf.plot(figsize=(10,10));

In [ ]:
gdf.head()

In [ ]:
# gdf['id'] = gdf['name'].apply(hash)

In [ ]:
countries = gdf.index

In [ ]:
n_rows = 48
n_sample = 40
assert n_sample < n_rows
dt_index = pd.date_range('2016-1-1', periods = n_rows, freq='M').strftime('%s')
max_color, min_color, max_opacity, min_opacity = 0,0,0,0

styledata = { }

for country in countries: 
    df = pd.DataFrame({'color': np.random.normal(size=n_rows), 
                       'opacity': np.random.normal(size=n_rows)},
                      index = dt_index)
    df = df.cumsum()
    df.sample(n_sample, replace=False).sort_index()
    styledata[country] = df
    


In [ ]:
df.plot()

In [ ]:
for country, data in styledata.items():
    max_color = max(max_color, data['color'].max())
    min_color = min(max_color, data['color'].min())
    max_opacity = max(max_color, data['opacity'].max())
    max_opacity = min(max_color, data['opacity'].max())

In [ ]:
cmap = linear.OrRd.scale(min_color, max_color)
norm = lambda x: (x - x.min())/(x.max()-x.min())
for country, data in styledata.items():
    data['color'] = data['color'].apply(cmap)
    data['opacity'] = norm(data['opacity'])

In [ ]:
# styledict = {str(country_id): data.to_dict(orient='index') for country_id, data in enumerate(styledata.values())}
styledict = {str(country): data.to_dict(orient='index') for country, data in styledata.items()}

In [ ]:
m = folium.Map((0,0), tiles='Stamen Toner', zoom_start=2)
g = TimeDynamicGeoJson(
    gdf.to_json(),
    styledict = styledict,
    style_function=lambda feature: {
        'color': 'white',
        'weight': 0.2,
        'dashArray': '5, 5',
        'fillOpacity': 1,
    },
    highlight_function=lambda feature: {
        'weight': 1,
        'color': '#666',
        'dashArray': '',
        'fillOpacity': 1
    }
).add_to(m)

m.save('timeslider.html')   
m

```python
styledict = {'japan': {
    '2017-1-1': {'color': 'ffffff', 'opacity': 1}
    '2017-1-2': {'color': 'fffff0', 'opacity': 1}
    ...
    }
}
```